COMP 215 - LAB 3 (NEO)
----------------
#### Name: Ben Blair
#### Date: 23/01/2023

As usual, the first code cell simply imports all the modules we'll be using...

In [38]:
import datetime, json, requests
from dataclasses import dataclass
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

## Fetch Complete Data for One Asteroid (from Lab 2)

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the asteroid with id '3014110'...


In [39]:
API_KEY = 'GT5a6VBQVVTnT8H1jSPfja2DfDMaC1d0UsYeWsYW'

id = '3014110'
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

{'absolute_magnitude_h': 21.3,
 'close_approach_data': [{'close_approach_date': '1901-01-23',
                          'close_approach_date_full': '1901-Jan-23 11:09',
                          'epoch_date_close_approach': -2175511860000,
                          'miss_distance': {'astronomical': '0.3113584635',
                                            'kilometers': '46578562.946072745',
                                            'lunar': '121.1184423015',
                                            'miles': '28942576.929564081'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '94290.7642054826',
                                                'kilometers_per_second': '26.191878946',
                                                'miles_per_hour': '58588.6140142345'}},
                         {'close_approach_date': '1908-09-20',
                          'close_approach_date_full': '1908-Sep-20 08:53',

## Lab 3 - Exercise 1
### Define an Asteroid Class

Using what we learned in the textbook, define a simple `Asteroid` class with some basic state variables for a single NEO.  Your Asteroid class should define at least 4 "state variables”:

*   `id`, `name`, `estimated_diameter`, and `is_potentially_hazardous`
*   Provide an `__init__(self, ...)` method to initialize a new Asteroid object with specific data values and a little code to test your class.
*   Add a `__str__(self)` method that returns a nicely formatted string representation of the object, plus a little code to test it.

In [40]:
@dataclass
class Asteroid:
    """ Represents an asteroid. """
    id: int
    name: str
    estimated_diameter: float
    is_potentially_hazardous: bool
    close_approaches: list

    # Replace __init__ and __str__ methods by using @dataclass
    # def __init__(self, id, name, diameter, hazardous):
    #     self.id = id
    #     self.name = name
    #     self.diameter = diameter
    #     self.hazardous = hazardous
    
    # def __str__(self):
    #     return f'Asteroid ID: {self.id}, Name: {self.name}, Est. Diameter (m): {self.diameter:.2f}, Hazardous (T/F): {self.hazardous}'

    @classmethod
    def from_NEO(cls, neo_id):
        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}/?api_key={API_KEY}'
        response = requests.request("GET", url, headers={}, data={})
        data = json.loads(response.text)

        name = data['name']
        max_diameter = float(data['estimated_diameter']['meters']['estimated_diameter_max'])
        min_diameter = float(data['estimated_diameter']['meters']['estimated_diameter_min'])
        estimated_diameter = (max_diameter + min_diameter) / 2
        is_potentially_hazardous = data['is_potentially_hazardous_asteroid']
        close_approaches = []

        asteroid = cls(neo_id, name, estimated_diameter, is_potentially_hazardous, close_approaches)
        asteroid.close_approaches = [CloseApproach.from_record(asteroid, approach) for approach in data['close_approach_data']]

        return asteroid

test_asteroid = Asteroid('1234567', '234567 2023 (BB31)', 555.55, True, [])
print(test_asteroid)

test_asteroid_from_id = Asteroid.from_NEO('3014110')
print(test_asteroid_from_id)

Asteroid(id='1234567', name='234567 2023 (BB31)', estimated_diameter=555.55, is_potentially_hazardous=True, close_approaches=[])
Asteroid(id='3014110', name='(1998 SU4)', estimated_diameter=236.3429308586, is_potentially_hazardous=True, close_approaches=[CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1901, 1, 23), miss_distance=46578562.94607274, relative_velocity=94290.7642054826), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1908, 9, 20), miss_distance=38539417.48119765, relative_velocity=103801.0019940824), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1912, 3, 20), miss_distance=64334023.67591313, relative_velocity=77694.2599215679), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1913, 9, 6), miss_distance=21602046.840450976, relative_velocity=65813.6151496408), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1917, 2, 5), 

## Lab 3 - Exercise 2
### Factory method: Asteroid.from_NEO

We want to be able to construct Asteroid objects easily from the NEO API.

Define a `“@classmethod”: from_NEO(cls, neo_id)` that takes the id for a single NEO, fetches the NEO record from the API, then constructs and returns an Asteroid object representing that NEO. This kind of method is sometimes called a “factory” – it constructs an object from raw materials.


In [41]:
# Add your code to the code cell in Exercise 1

## Lab 3 - Exercise 3
### Define a CloseApproach class

Each NEO comes with a list of “`close_approach_data`”, where each record in this list represents a single “close approach” to another orbiting body.

*   Develop a class named “**CloseApproach**” to represent a single close approach record. State variables are **asteroid** (Asteroid object),  **orbiting body** (`str`), **approach date** (`datetime` object!),  **miss distance** (`float` choose units, document it, and be consistent!),  and **relative velocity** (ditto).
*   Define a Factory class method to construct a `CloseApproach` object from one close approach data record (a dictionary object).

Remember to parse the date/time string into a `datetime` object.

In [42]:
@dataclass
class CloseApproach:
    """ Represents a single close approach record. """
    asteroid: Asteroid
    orbiting_body: str
    approach_date: datetime
    miss_distance: float
    relative_velocity: float

    @classmethod
    def from_record(cls, asteroid, record):
        body = record['orbiting_body']
        date = datetime.datetime.strptime(record['close_approach_date'], '%Y-%m-%d').date()
        distance = float(record['miss_distance']['kilometers'])
        velocity = float(record['relative_velocity']['kilometers_per_hour'])

        return cls(asteroid, body, date, distance, velocity)

test_approach = {
    'close_approach_date': '1900-10-30',
    'close_approach_date_full': '1900-Oct-30 21:14',
    'epoch_date_close_approach': -2182819560000,
    'miss_distance': {
        'astronomical': '0.4183490942',
        'kilometers': '62584133.408749354',
        'lunar': '162.7377976438',
        'miles': '38887977.2407309252'},
    'orbiting_body': 'Earth',
    'relative_velocity': {
        'kilometers_per_hour': '30027.9669430086',
        'kilometers_per_second': '8.3411019286',
        'miles_per_hour': '18658.2109041156'}}

CloseApproach.from_record('3014110', test_approach)

CloseApproach(asteroid='3014110', orbiting_body='Earth', approach_date=datetime.date(1900, 10, 30), miss_distance=62584133.40874936, relative_velocity=30027.9669430086)

## Lab 3 - Exercise 4
### Add list of close approaches to Asteroid

Every Asteroid should have a list of “close approaches”.  Add a new state variable to your Asteroid class, initially an empty list.  In the factory method, use a list comprehension to build a list of `CloseApproach` objects for the Asteroid from the NEO data record.

Extend your test code to demonstrate this new feature.

In [43]:
id = test_asteroid_from_id.id
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

# pprint(data)

# Test list comprehension before adding it into the Asteroid class factory method
close_approaches = [CloseApproach.from_record(test_asteroid_from_id, approach) for approach in data['close_approach_data']]

pprint(close_approaches)

[CloseApproach(asteroid=Asteroid(id='3014110', name='(1998 SU4)', estimated_diameter=236.3429308586, is_potentially_hazardous=True, close_approaches=[CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1901, 1, 23), miss_distance=46578562.94607274, relative_velocity=94290.7642054826), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1908, 9, 20), miss_distance=38539417.48119765, relative_velocity=103801.0019940824), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1912, 3, 20), miss_distance=64334023.67591313, relative_velocity=77694.2599215679), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1913, 9, 6), miss_distance=21602046.840450976, relative_velocity=65813.6151496408), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1917, 2, 5), miss_distance=39908236.52563852, relative_velocity=78525.9998454341), CloseApproach(asteroid=..., orbitin

## Lab 3 - Optional
### Take your skills to the next level!

With this data structure in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.

Here’s a couple ideas to try:

*   Add a  method to the `Asteroid` class, `closest_earth_approach`, that returns the `CloseApproach` object that represents the closest approach the Asteroid makes to Earth.

*   Write a function named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth in within that range. Your algorithm will ultimately need to:

   *   grab the list of NEO’s for the given date range
   *   use a list comprehension to build the list of Asteroid objects for the NEOs returned
   *   use a list comprehension to **filter** potentially hazardous Asteroids only
   *   use a list comprehension to **map** each Asteroid to its `closest_earth_approach`
   *   apply Python’s `min` function to identify the Asteroid with the `closest_earth_approach`

   You may want to *decompose* some of these steps into smaller functions.

*   Add a method to the `Asteroid` class, `estimated_mass`, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.

*   Add a method to the `CloseApproach` class, `impact_force`,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [ ]:
# Add a closest_earth_approach method



In [ ]:
# Write a function most_dangerous_approach



In [ ]:
# Add an estimated_mass method



In [ ]:
# Add an impact_force method

